In [18]:
#Import Library
import string
import pandas as pd
import numpy as np

In [19]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# IMPORTING DATASET
Dataset didapatkan menggunakan teknik web scraping dari website google playstore dengan Google Scraper
> library pandas digunakan untuk menampilkan dan membaca dataset

In [34]:
data = pd.read_csv("/content/gdrive/MyDrive/Analisis Sentimen/dataset/ulasan-sentiment.csv", sep=",", encoding="latin1")
data.head(10)

,Review,sentiment
0,Kenapa kasih rate jelek? :( Developer ini memb...,1
1,Saya merasa bahwa aplikasi berjalan dengan lam...,-1
2,aku baru download sih.... msh blm genap 3kg. t...,1
3,Aplikasinya keren bisa membantu supaya dapat d...,1
4,"Sangat berharap bisa buka app ini, tp setelah ...",1
5,"Sering nya nggak ada yg pick up, jadi kita har...",1
6,"Gak bisa registnya, ketika telephone masuk oto...",-1
7,"Aplikasinya bagus, semoga secepatnya bisa dike...",1
8,Harga sampah plastiknya terlalu murah banget. ...,-1
9,Memutuskan untuk meng-uninstall aplikasi ini. ...,-1


# PROSES CLEANING DATA
> Case Folding = mengambil string sebagai input mengubahnya menjadi huruf kecil dengan menggunakan metode lower

In [37]:
def casefolding(Review):
  Review = Review.lower()
  return Review
data['Review'] = data['Review'].apply(casefolding)
data.head(10)

,Review,sentiment
0,kenapa kasih rate jelek? :( developer ini memb...,1
1,saya merasa bahwa aplikasi berjalan dengan lam...,-1
2,aku baru download sih.... msh blm genap 3kg. t...,1
3,aplikasinya keren bisa membantu supaya dapat d...,1
4,"sangat berharap bisa buka app ini, tp setelah ...",1
5,"sering nya nggak ada yg pick up, jadi kita har...",1
6,"gak bisa registnya, ketika telephone masuk oto...",-1
7,"aplikasinya bagus, semoga secepatnya bisa dike...",1
8,harga sampah plastiknya terlalu murah banget. ...,-1
9,memutuskan untuk meng-uninstall aplikasi ini. ...,-1


In [39]:
# Proses Cleansing remove regex (cleansing) seperti tanda baca dan angka angka
import re
import string
def cleansing(ulasan) :
  ulasan = ulasan.strip(" ")
  ulasan = re.sub(r'[?|$|.|!_:")(-+,]', '', ulasan)
  ulasan = re.sub(r"\b[a-aA-Z]\b", "", ulasan)
  ulasan = re.sub('\s+',' ', ulasan)
  return ulasan
data['Review'] = data['Review'].apply(cleansing)
data.head(10)

,Review,sentiment
0,kenapa kasih rate jelek developer ini membantu...,1
1,saya merasa bahwa aplikasi berjalan dengan lam...,-1
2,aku baru download sih msh blm genap 3kg tp aku...,1
3,aplikasinya keren bisa membantu supaya dapat d...,1
4,sangat berharap bisa buka app ini tp setelah l...,1
5,sering nya nggak ada yg pick up jadi kita haru...,1
6,gak bisa registnya ketika telephone masuk otom...,-1
7,aplikasinya bagus semoga secepatnya bisa dikem...,1
8,harga sampah plastiknya terlalu murah banget s...,-1
9,memutuskan untuk meng-uninstall aplikasi ini a...,-1


In [40]:
# Library utuk memproses dan menganalisis teks bahasa alami (termasuk tokenisasi, stemming, lemmatization, analisis sentimen, dan lainnya)
pip install nltk

In [42]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

# NLTK word tokenize -> tokenize untuk memecah teks menjadi kata kata
def word_tokenize_wrapper(text):
  return word_tokenize(text)

data['Review'] = data['Review'].apply(word_tokenize_wrapper)
data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,Review,sentiment
0,"[kenapa, kasih, rate, jelek, developer, ini, m...",1
1,"[saya, merasa, bahwa, aplikasi, berjalan, deng...",-1
2,"[aku, baru, download, sih, msh, blm, genap, 3k...",1
3,"[aplikasinya, keren, bisa, membantu, supaya, d...",1
4,"[sangat, berharap, bisa, buka, app, ini, tp, s...",1


In [43]:
# Setelah cleansing tokenize maka data disimpan dengan
data.to_csv('normalisasi.csv', index=False)

# Normalization
menghindari masalah seperti dominasi fitur dengan skala nilai yang besar.
> normalizad_word_dict digunakan untuk menyimpan bentuk-bentuk yang dinormalisasi sebagai kunci nilai

> normalized_term didefinisikan dalam code mengambil kata sebagai masukan dan mengganti setiap kata dalam bentuk normalisasi jika ada di dalam kamus normalisasi word dict

In [45]:
normalizad_word = pd.read_csv("normalisasi.csv")

normalizad_word_dict = {}

for index, row in normalizad_word.iterrows():
  if row[0] not in normalizad_word_dict:
    normalizad_word_dict[row[0]] = row[1]

def normalized_term(document):
  return [normalizad_word_dict[term] if term in normalizad_word_dict else term for term in document]

  data['Review'] = data['Review'].apply(normalized_term)

  data['Review'].head(20)

In [46]:
# simpan data setelah dinormalisasi
data.to_csv('stopwords.csv', index=False)

# STOPWORDS
kata kata umum yang tidak memiliki banyak arti, yang sering dihilangkan dari data teks untuk meningkatkan akurasi analisis.
> Pustaka Stopwords diambil dari nltk.corpus, digunakan untuk mendapatkan daftatr stopwords dalam bahasa Id dan Eng

> Stopwords Removal -> untuk mendefinisian dalam code, mengambil daftar kata sebagai masukan dan menghapus kata2 yang ada dalam daftar stopwords

In [47]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw = pd.read_csv("stopwords.csv")
def stopword_removal(ulasan):
  filtering = stopwords.words('indonesian', 'english')
  filtering.extend(sw)
  x = []
  data = []
  def myFunc(x):
    if x in filtering:
      return False
    else:
      return True
  fit = filter(myFunc, ulasan)
  for x in fit:
    data.append(x)
  return data
data['Review'] = data['Review'].apply(stopword_removal)
data.head(10)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,Review,sentiment
0,"[kasih, rate, jelek, developer, membantu, utk,...",1
1,"[aplikasi, berjalan, lambat, masuk, penentuan,...",-1
2,"[download, sih, msh, blm, genap, 3kg, tp, sena...",1
3,"[aplikasinya, keren, membantu, didaur, ulang, ...",1
4,"[berharap, buka, app, tp, log, in, applikasi, ...",1
5,"[nya, nggak, yg, pick, up, chat, langsung, via...",1
6,"[gak, registnya, telephone, masuk, otomatis, s...",-1
7,"[aplikasinya, bagus, semoga, secepatnya, dikem...",1
8,"[harga, sampah, plastiknya, murah, banget, mur...",-1
9,"[memutuskan, meng-uninstall, aplikasi, aplikas...",-1


# STEMMING
Mereduksi kata menjadi bentuk dasar/kata dasar untuk meningkatan akurasi analisis
> Mengambil pustaka Sastrawi yang digunakan untuk membuat objek stemmer untuk stemming pada setiap kata pada kolom review pada dataframe

In [48]:
# proses stemming
! pip install Sastrawi

In [51]:
from sklearn.pipeline import Pipeline
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def stemming(ulasan):
  factory = StemmerFactory()
  stemmer = factory.create_stemmer()
  do = []
  for w in ulasan:
    dt = stemmer.stem(w)
    do.append(dt)
  d_clean = []
  d_clean = " ".join(do)
  print(d_clean)
  return d_clean
data['Review'] = data['Review'].apply(stemming)

# Simpan data yang sudah bersih
data.to_csv('duitkubersih.csv', index=False)
data_clean = pd.read_csv('duitkubersih.csv', encoding='latin1')
data_clean.head

kasih rate jelek developer bantu utk mudah dlm daur ulang dgn aplikasi contact via wa direspon kadang slow res request by wa picker jemput lokasi sdu timbang masuk aplikasi manual aplikasi 100  sempurna developer bantu rugi sabar
aplikasi jalan lambat masuk tentu lokasi pengambilang brg klik cek tentu sampah tulis apa dr segi ui ux desain icon card product font baik menu navigator padat sbg guna yg kali pakai aplikasi mata lelah saking padat layout baik aplikasi
download sih msh blm genap 3kg tp senang aplikasi kayak gin bantu banget smg depan byk org biasa mem sampah rumah tdk buang sampah sembarang
aplikasi keren bantu daur ulang  duit dr sampah yg proses aman ramah  cepat saran aja kaleng2 palstik2 moga angkut  duitin
harap buka app tp log in applikasi gak ngerti knp udh coba dihp dgn akun yg tp langsung appnya
nya nggak yg pick up chat langsung via wa cs duitin overall aplikasi fungsi ngitung nilai sampah baca artikel dll
gak registnya telephone masuk otomatis sinyal 4g internet he

<bound method NDFrame.head of                                                 Review  sentiment
0    kasih rate jelek developer bantu utk mudah dlm...          1
1    aplikasi jalan lambat masuk tentu lokasi penga...         -1
2    download sih msh blm genap 3kg tp senang aplik...          1
3    aplikasi keren bantu daur ulang  duit dr sampa...          1
4    harap buka app tp log in applikasi gak ngerti ...          1
..                                                 ...        ...
716                                               bgus          1
717                                             mantap          1
718                                            bintang          1
719                                                top          1
720                                              tarik          1

[721 rows x 2 columns]>

In [75]:
import string
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import re
data_clean = pd.read_csv('duitkubersih.csv', encoding='latin1')
data_clean.head()

,Review,sentiment
0,kasih rate jelek developer bantu utk mudah dlm...,1
1,aplikasi jalan lambat masuk tentu lokasi penga...,-1
2,download sih msh blm genap 3kg tp senang aplik...,1
3,aplikasi keren bantu daur ulang duit dr sampa...,1
4,harap buka app tp log in applikasi gak ngerti ...,1


In [55]:
# Mengubah tipe data
data_clean = data_clean.astype({'sentiment' : 'category'})
data_clean = data_clean.astype({'Review' : 'string'})
data_clean.dtypes

Review         string
sentiment    category
dtype: object

# Proses TF IDF
Statistik numerik yang mencerminkan seberapa penting sebuah kata untuk sebuah dokumen dalam koleksi/corpus

In [58]:
# Proses TF IDF
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = CountVectorizer()
X = vectorizer.fit_transform(data_clean['Review'].astype('U'))

tf = TfidfVectorizer()
text_tf = tf.fit_transform(data_clean['Review'].astype('U'))
print(text_tf)

  (0, 1283)	0.16307787991598055
  (0, 1279)	0.16917067954961945
  (0, 1355)	0.18809664385780037
  (0, 0)	0.18809664385780037
  (0, 919)	0.18809664385780037
  (0, 925)	0.09615466893419462
  (0, 1518)	0.153890709110723
  (0, 1319)	0.16917067954961945
  (0, 880)	0.1441519156077996
  (0, 661)	0.13698821813586787
  (0, 1172)	0.11239278662507662
  (0, 284)	0.18809664385780037
  (0, 1259)	0.17702566444752751
  (0, 1260)	0.18809664385780037
  (0, 1390)	0.18809664385780037
  (0, 687)	0.18809664385780037
  (0, 407)	0.18809664385780037
  (0, 1619)	0.28309905765011845
  (0, 1616)	0.16307787991598055
  (0, 332)	0.18809664385780037
  (0, 103)	0.19847317564722247
  (0, 383)	0.15809970013934657
  (0, 1587)	0.099136289519506
  (0, 362)	0.10994598086072224
  (0, 415)	0.17702566444752751
  :	:
  (707, 362)	0.21704342216686587
  (708, 915)	0.7679037233520637
  (708, 1051)	0.6405652750985159
  (709, 453)	0.5571724794211038
  (709, 1510)	0.5571724794211038
  (709, 884)	0.5571724794211038
  (709, 443)	0.2620

# Proses Cosin Similarity
> dari modul pairwise pada paket matriks diperpustakaan sklearn, fungsi ini digunakan untuk menghitung kemiripan cosine antara dua set data

In [59]:
# Import cosine_simlarity metrics
from sklearn.metrics.pairwise import cosine_similarity

In [67]:
# compute similarity using cosine similarity
# menghitung kemiripan cosine antara baris matriks text_tf, matriks cosine berisikan nilai kemiripan cosine berpasangan antar baris matrik text_tf
# setiap baris mewakili dokumen dan setiap kolom mewakili kata dlam kosakata
cos_sim = cosine_similarity(text_tf, text_tf)

print(cos_sim)

[[1.         0.06282825 0.04419771 ... 0.         0.         0.        ]
 [0.06282825 1.         0.03629255 ... 0.         0.         0.        ]
 [0.04419771 0.03629255 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


# SPLITTING DATA
Membagi data ke dalam bentuk set Pelatihan dan Pengujian menggunakan fungsi train_test_split
> matriks cos_sim diatas dan kolom sentimen dari data clean masing-masing digunakan sebagai fitur input dan variabel target.

> untuk parameter test_size diatur 0.2 yang berarti 20% data digunakan untuk pengujian dan 80% sisanya digunakan untu pelatihan

> parameter random_state diatur 33 untuk memastikan kekonsistenan hasil

>> selain itu code ini digunakan untuk mencetak informasi tentang dsitribusi label sentiman di setiap set

In [76]:
# splitting data
import collections, numpy
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cos_sim, data_clean['sentiment'], test_size = 0.2, random_state = 33)
print("Jumlah Data Uji :", X_test.shape)
print("Jumlah Data Latih :", X_train.shape)

pos = (y_test == 1).sum()
net = (y_test == 0).sum()
neg = (y_test == -1).sum()
postrain = (y_train == 1).sum()
nettrain = (y_train == 0).sum()
negtrain = (y_train == -1).sum()
total = pos + neg
print("Jumlah data uji dengan sentimen positif:", pos)
print("Jumlah data uji dengan sentimen netral:", net)
print("Jumlah data uji dengan sentimen negatif:", neg)
print("Jumlah data latih dengan sentimen positif:", postrain)
print("Jumlah data latih dengan sentimen netral:", nettrain)
print("Jumlah data latih dengan sentimen negatif:", negtrain)
data_clean['sentiment'].value_counts()

Jumlah Data Uji : (145, 721)
Jumlah Data Latih : (576, 721)
Jumlah data uji dengan sentimen positif: 85
Jumlah data uji dengan sentimen netral: 26
Jumlah data uji dengan sentimen negatif: 34
Jumlah data latih dengan sentimen positif: 304
Jumlah data latih dengan sentimen netral: 95
Jumlah data latih dengan sentimen positif: 177


 1    389
-1    211
 0    121
Name: sentiment, dtype: int64

# Perform Algoritma KNN
>> fit digunakan untuk melatih model pada data training
>> predict digunakan untu membuat prediksi pada data testing (mencetak akurasi, presisi, recall, f1 score, matriks keracuan dan laporan klasifikasi)

> Confusion matriks menunjukan jumlah true positif dan true negatif, false positif, false negatif
> Clasification_report menunjukan berbagai matriks performa setiap precision(recall dan f1 score) untuk setiap kelas

In [81]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

In [83]:
# Langkah 1: Melatih model
clf = KNeighborsClassifier(n_neighbors=7).fit(X_train, y_train)

# Langkah 2: Membuat prediksi
predicted = clf.predict(X_test)


In [84]:
print(classification_report(y_test, predicted, labels=[1, 0, -1], zero_division=0))

# ...

print("Precision (POSITIF):", precision_score(y_test, predicted, average="macro", labels=[1, 0, -1]))
print("Recall (POSITIF):", recall_score(y_test, predicted, average="macro", labels=[1, 0, -1]))

              precision    recall  f1-score   support

           1       0.78      0.69      0.73        85
           0       0.12      0.04      0.06        26
          -1       0.41      0.74      0.53        34

    accuracy                           0.59       145
   macro avg       0.44      0.49      0.44       145
weighted avg       0.57      0.59      0.56       145

Precision (POSITIF): 0.4370506183491516
Recall (POSITIF): 0.489291101055807


In [89]:
# perform algoritma dengan
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=7).fit(X_train, y_train)
predicted = clf.predict(X_test)
print(f'confusion matrix:\n {confusion_matrix(y_test, predicted)}')
print('===============================================\n')
tn, fp, fn, tp = confusion_matrix(y_test, predicted).ravel()
print("TN:", tn)
print("FP:", fp)
print("FN:", fn)
print("TP:", tp)
print(classification_report(y_test, predicted, zero_division=0))
print('===============================================\n')
print("Hasil Klasifikasi Sentimen Analisis Duitin:")
print("knn Accuracy:", accuracy_score(y_test, predicted))
print("knn Precision:", precision_score(y_test, predicted, average="macro", pos_label="NEGATIF"))
print("knn Recall:", recall_score(y_test, predicted, average="macro", pos_label="NEGATIF"))
print("knn f1_score:", f1_score(y_test, predicted, average="macro", pos_label="NEGATIF"))

print(f'confusion matrix:\n {confusion_matrix(y_test, predicted)}')
print('===============================================\n')
print(classification_report(y_test, predicted, zero_division=0))

knn Accuracy: 0.5862068965517241
knn Precision: 0.4370506183491516
knn Recall: 0.489291101055807
knn f1_score: 0.43935285784794464
confusion matrix:
 [[25  1  8]
 [16  1  9]
 [20  6 59]]

              precision    recall  f1-score   support

          -1       0.41      0.74      0.53        34
           0       0.12      0.04      0.06        26
           1       0.78      0.69      0.73        85

    accuracy                           0.59       145
   macro avg       0.44      0.49      0.44       145
weighted avg       0.57      0.59      0.56       145



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'NEGATIF') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'NEGATIF') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'NEGATIF') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [95]:
# perform algoritma KNN
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=7).fit(X_train, y_train)
predicted = clf.predict(X_test)
print(f'confusion matrix:\n {confusion_matrix(y_test, predicted)}')
print('===============================================\n')
conf_matrix = confusion_matrix(y_test, predicted, labels=[1, 0, -1])

print(classification_report(y_test, predicted, zero_division=0))
print('===============================================\n')
print("Hasil Klasifikasi Sentimen Analisis Duitin:")
print("Accuracy:", accuracy_score(y_test, predicted))
print("Precision:", precision_score(y_test, predicted, average="macro", pos_label="POSITIF"))
print("Recall:", recall_score(y_test, predicted, average="macro", pos_label="POSITIF"))
print("error_rate:", 1-accuracy_score(y_test, predicted))

confusion matrix:
 [[25  1  8]
 [16  1  9]
 [20  6 59]]

              precision    recall  f1-score   support

          -1       0.41      0.74      0.53        34
           0       0.12      0.04      0.06        26
           1       0.78      0.69      0.73        85

    accuracy                           0.59       145
   macro avg       0.44      0.49      0.44       145
weighted avg       0.57      0.59      0.56       145


Hasil Klasifikasi Sentimen Analisis Duitin:
Accuracy: 0.5862068965517241
Precision: 0.4370506183491516
Recall: 0.489291101055807
error_rate: 0.4137931034482759


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'POSITIF') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'POSITIF') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(


In [100]:
from sklearn import model_selection
# splitting data 80:20
# matrix cos_sim dan kolom sentiment dari dataframe masing-masing digunakan sebagai fitur input dan variabel target
import collections, numpy
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(cos_sim, data_clean['sentiment'], test_size= 0.2, random_state = 33)
print("Jumlah Data Uji : ", X_test.shape)
print("Jumlah Data Latih : ", X_train.shape)

pos = (y_test == 1).sum()
net = (y_test == 0).sum()
neg = (y_test == -1).sum()
postrain = (y_train == 1).sum()
nettrain = (y_train == 0).sum()
negtrain = (y_train == -1).sum()
total = pos + neg
print("Jumlah data uji dengan sentimen positif:", pos)
print("Jumlah data uji dengan sentimen netral:", net)
print("Jumlah data uji dengan sentimen negatif:", neg)
print("Jumlah data latih dengan sentimen positif:", postrain)
print("Jumlah data latih dengan sentimen netral:", nettrain)
print("Jumlah data latih dengan sentimen negatif:", negtrain)
data_clean['sentiment'].value_counts()

Jumlah Data Uji :  (145, 721)
Jumlah Data Latih :  (576, 721)
Jumlah data uji dengan sentimen positif: 85
Jumlah data uji dengan sentimen netral: 26
Jumlah data uji dengan sentimen negatif: 34
Jumlah data latih dengan sentimen positif: 304
Jumlah data latih dengan sentimen netral: 95
Jumlah data latih dengan sentimen negatif: 177


 1    389
-1    211
 0    121
Name: sentiment, dtype: int64

In [101]:
# perform algoritma dengan
# mencetak confusion matriks dan laporan klasifikasi yang memberikan informasi kinerja model pada data pengujian
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier

clf = KNeighborsClassifier(n_neighbors=7).fit(X_train, y_train)
predicted = clf.predict(X_test)

print("knn Accuracy:", accuracy_score(y_test, predicted))
print("knn Precision:", precision_score(y_test, predicted, average="macro", pos_label="NEGATIF"))
print("knn Recall:", recall_score(y_test, predicted, average="macro", pos_label="NEGATIF"))
print("knn f1_score:", f1_score(y_test, predicted, average="macro", pos_label="NEGATIF"))

print(f'confusion matrix:\n {confusion_matrix(y_test, predicted)}')
print('===============================================\n')
print(classification_report(y_test, predicted, zero_division=0))

knn Accuracy: 0.5862068965517241
knn Precision: 0.4370506183491516
knn Recall: 0.489291101055807
knn f1_score: 0.43935285784794464
confusion matrix:
 [[25  1  8]
 [16  1  9]
 [20  6 59]]

              precision    recall  f1-score   support

          -1       0.41      0.74      0.53        34
           0       0.12      0.04      0.06        26
           1       0.78      0.69      0.73        85

    accuracy                           0.59       145
   macro avg       0.44      0.49      0.44       145
weighted avg       0.57      0.59      0.56       145



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'NEGATIF') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'NEGATIF') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1396: UserWarning: Note that pos_label (set to 'NEGATIF') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
